In [ ]:
import numpy as np

import healpix_convolution as hc
from healpix_convolution.convolution import convolve
from healpix_convolution.plotting import plot_healpix

## load data

In [ ]:
import xarray as xr

url = "/home/jmagin/work/data/destine/average_surface_temperature.zarr"
ds = xr.open_dataset(url, engine="zarr", chunks={}).isel(oceanModelLayer=0)
ds

In [ ]:
computed = ds.compute()
data = computed["avg_thetao"].data
cell_ids = computed["cell_ids"].data
resolution = computed["cell_ids"].attrs["resolution"]
indexing_scheme = computed["cell_ids"].attrs["indexing_scheme"]

## compute the kernel

In [ ]:
%%time
kernel = hc.kernels.gaussian_kernel(
    cell_ids,
    resolution=resolution,
    indexing_scheme=indexing_scheme,
    sigma=0.001,
    truncate=3,
)
kernel

## convolve the data

In [ ]:
%%time
# `axis` does not work, the spatial dimension needs to be last
# use `numpy.moveaxis` if it is not already the last dimension
result = convolve(data, kernel)
result

## plot the result

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig, axes = plt.subplots(
    ncols=1, nrows=2, figsize=(16, 14), subplot_kw={"projection": ccrs.Mollweide()}
)
plot_healpix(
    data,
    cell_ids,
    ax=axes[0],
    resolution=resolution,
    cmap="plasma",
    title="original SST",
)
plot_healpix(
    result,
    cell_ids,
    ax=axes[1],
    resolution=resolution,
    cmap="plasma",
    title="result of the convolution",
)